In [1]:
from classes.AlexNet import AlexNet

import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models

# Load AlexNet

## First get AlexNet pretrained model

**Prepare data**

In [2]:
# Transformation steps for input data 
AlexTransform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Path to mnist data
mnist_data_path = os.path.join("..", "..", "data", 'mnist-data')

# Create a loader for training data and testing data
batch_size = 500
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(mnist_data_path, train=True, download=True, transform=AlexTransform),
        batch_size=batch_size, shuffle=True)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(mnist_data_path, train=False, download=True, transform=AlexTransform),
        batch_size=1, shuffle=False)

val_loader = torch.utils.data.DataLoader(
        datasets.MNIST(mnist_data_path, train=False, download=True, transform=AlexTransform),
        batch_size=batch_size, shuffle=False)

In [3]:
# Check Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


**Get AlexNet Model**

**Pre-trained from PyTorch**

In [4]:
# # Get pretrained model with default weights
# net = models.alexnet(weights=models.AlexNet_Weights.DEFAULT)

# # Change last layer to reflect MNIST categories
# net_fc = net.classifier[-1] 
# num_ftrs = net_fc.in_features # Get the amount of input features for the last layer

# # Update to 10 possible outputs
# net.classifier[-1] = nn.Linear(num_ftrs, 10)

**Pre-trained from scratch**

In [5]:
net = AlexNet(num_classes=10)

**Load weights from trained model**

In [6]:
net_path = os.path.join("..","..","data", "AlexNet", "AlexNet_scratch_MNIST.pth")
net.load_state_dict(torch.load(net_path, weights_only=True))
net.eval()

AlexNet(
  (layer1): Sequential(
    (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (layer4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (layer5): Sequential(
    (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU()
  )
  (fc1): 

# Prediction time baby

In [7]:
# Get image from test_loader
image, label = next(iter(test_loader))

# Get prediction
softmax = nn.Softmax(dim=1)
pred_raw = net(image)
pred_probs = softmax(pred_raw)
print("Raw output: ", pred_raw.detach().numpy())
print("Softmax output: ", pred_probs.detach().numpy())


Raw output:  [[ -1.1507683   -0.5124087    2.9930816    4.2085643   -5.2940187
   -2.0705087  -14.564747    16.028841    -0.22747564   1.5017831 ]]
Softmax output:  [[3.4592823e-08 6.5497062e-08 2.1809083e-06 7.3538504e-06 5.4902843e-10
  1.3789475e-08 5.1685554e-14 9.9998975e-01 8.7089688e-08 4.9087913e-07]]
